In [21]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

using DataFrames, DataFramesMeta
using CSV
using Statistics
using ScikitLearn
using Main

In [22]:
@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [23]:
KEY = "2019week0"
YEAR = "2019"
COLUMN = :totalPoints

:totalPoints

In [24]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 58×54 DataFrame. Omitted printing of 50 columns
│ Row │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│     │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├─────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1   │ 0            │ 9          │ Panel         │ Panel         │
│ 2   │ 0            │ 12         │ Panel         │ PanelAndCargo │
│ 3   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 4   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 5   │ 1            │ 0          │ Panel         │ Panel         │
│ 6   │ 0            │ 0          │ Panel         │ Panel         │
│ 7   │ 1            │ 0          │ Panel         │ Panel         │
│ 8   │ 0      

In [25]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

scores_test = @linq scores |> where(:level .!= "qm")
teams_test = @linq teams |> where(:level .!= "qm")

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train, shortevent=true)
y = convert.(Float64, coalesce(scores_train[COLUMN], -1.0))

26-element Array{Float64,1}:
  9.0
  3.0
 33.0
 28.0
 24.0
 24.0
 31.0
 34.0
 43.0
 20.0
 12.0
 23.0
 15.0
 23.0
  7.0
 31.0
 21.0
  9.0
 26.0
 47.0
 21.0
 15.0
 27.0
 25.0
 11.0
 18.0

In [26]:
model1 = ScikitLearn.fit!(LinearRegression(fit_intercept=false), ß, y)

PyObject LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [27]:
oprs = DataFrame(hcat(teamnums, model1[:coef_]), [:team, :rating])
#sort(oprs, (:rating), rev = true) |> print

,team,rating
,Any,Any
1,frc1073,-8.57041
2,frc1153,15.3282
3,frc1517,12.2336
4,frc1721,-0.974181
5,frc1729,19.4517
6,frc190,-4.08008
7,frc2084,7.73346
8,frc2342,14.5362
9,frc246,10.5781


In [28]:
model2 = ScikitLearn.fit!(ElasticNet(alpha=0, l1_ratio=0, fit_intercept=true), ß, y)
oprs2 = DataFrame(hcat(teamnums, model2[:coef_]), [:team, :rating])
sort(oprs2, (:rating), rev = true) |> print

22×2 DataFrame
│ Row │ team    │ rating   │
│     │ Any     │ Any      │
├─────┼─────────┼──────────┤
│ 1   │ frc3236 │ 17.1412  │
│ 2   │ frc1729 │ 12.2261  │
│ 3   │ frc58   │ 11.2839  │
│ 4   │ frc501  │ 10.8639  │
│ 5   │ frc1153 │ 8.10263  │
│ 6   │ frc2342 │ 7.3106   │
│ 7   │ frc811  │ 5.37237  │
│ 8   │ frc1517 │ 5.00804  │
│ 9   │ frc246  │ 3.35255  │
│ 10  │ frc6153 │ 3.15578  │
│ 11  │ frc5687 │ 2.47237  │
│ 12  │ frc7314 │ 1.15161  │
│ 13  │ frc2084 │ 0.507864 │
│ 14  │ frc5813 │ -1.20264 │
│ 15  │ frc4905 │ -5.62552 │
│ 16  │ frc3467 │ -8.12396 │
│ 17  │ frc1721 │ -8.19978 │
│ 18  │ frc190  │ -11.3057 │
│ 19  │ frc509  │ -11.7793 │
│ 20  │ frc3986 │ -14.3653 │
│ 21  │ frc1073 │ -15.796  │
│ 22  │ frc3323 │ -16.4253 │

In [20]:
(ßtest, _, _) = Lib.opr_ß(scores_test, teams_test, teamnums)

KeyError: KeyError: key "frc9998" not found

In [130]:
test_y = convert.(Float64, coalesce(scores_test[COLUMN], -1.0))
test_yhat_m1 = ScikitLearn.predict(model1, ßtest)
test_yhat_m2 = ScikitLearn.predict(model2, ßtest)

200-element Array{Float64,1}:
 356.4634845230824 
 469.8945103790024 
 356.4634845230824 
 469.8945103790024 
 478.4038875502631 
 396.2738478142097 
 445.924253341976  
 396.2738478142097 
 445.924253341976  
 396.2738478142097 
 356.4634845230824 
 413.23055699752   
 356.4634845230824 
   ⋮               
 466.47721702579304
 449.008953579127  
 466.47721702579304
 449.008953579127  
 466.66331521952486
 386.894678606519  
 466.66331521952486
 386.894678606519  
 434.2987333174266 
 466.47721702579304
 488.1537770916406 
 466.47721702579304

In [131]:
predictions_table = DataFrame(hcat(scores_test.key, test_y, test_yhat_m1, test_yhat_m2), [:key, :y, :LinReg, :RidgeReg])

,key,y,LinReg,RidgeReg
,Any,Any,Any,Any
1,2018arc_f1m1_red,356.0,356.463,356.463
2,2018arc_f1m1_blue,361.0,469.895,469.895
3,2018arc_f1m2_red,356.0,356.463,356.463
4,2018arc_f1m2_blue,397.0,469.895,469.895
5,2018arc_qf1m1_red,270.0,478.404,478.404
6,2018arc_qf1m1_blue,317.0,396.274,396.274
7,2018arc_qf1m2_red,455.0,445.924,445.924
8,2018arc_qf1m2_blue,303.0,396.274,396.274
9,2018arc_qf1m3_red,443.0,445.924,445.924


In [132]:
rmse_1 = mean((predictions_table.LinReg-predictions_table.y).^2) |> sqrt

128.15965838569656

In [133]:
rmse_2 = mean((predictions_table.RidgeReg-predictions_table.y).^2) |> sqrt

128.15965838569633

UndefVarError: UndefVarError: ß not defined